In [3]:
import numpy as np
import pandas as pd
import math
import re
import sys
from shapely.geometry import Polygon
from matplotlib import pyplot as plt
from collections import Counter, OrderedDict, namedtuple, defaultdict, ChainMap
from queue import Queue
from copy import deepcopy
import networkx as nx
from functools import cmp_to_key, reduce
from itertools import product, permutations, combinations, combinations_with_replacement
from itertools import repeat
from functools import cache
from scipy.sparse import csr_matrix
from scipy.sparse.csgraph import maximum_flow
import json
import time

In [4]:
sys.setrecursionlimit(1500)

In [5]:
def open_input(day='04', suffix="input"):
    with open(f"{day}-{suffix}", "r") as file:
        lines = file.readlines()
    data_raw = [line.replace("\n", "") for line in lines]
    data_raw = "\n".join(data_raw)
    return data_raw

input_raw = open_input()
input_raw

'..@..@..@..@..@@..@@@@@@.@.@@@@.@.@@@..@.@@@.@@@.@@@.@@@.@.@.@@..@@@@@@@@@@.@@@..@@.@@@@..@@.@.@@..@@@@@@@@...@.@@.@@.@@@@.@@.@@......@@.@@\n@@@@@@@@@@@..@.@@@@.@...@@.@@@.@@@@@@@@.@.@@...@@..@@..@@..@.@..@.@.@@..@.@@@@@@@@@@@.@@@@@@.@@.....@@@@..@@.@..@@@@@@@@.@.@.@@@...@@@.@@@@\n.@@@.@@..@@@@.@.@..@@.....@....@@@@.@@@.@...@.@@.@.@@@@@@@@@@@@@@..@@@@@.@@@.@.@@...@@.@@.@@@@@..@..@@@@@@..@@@@...@@@.@.@@..@.@@@@.@.@@.@@\n@...@@@.@@.@@@.@@@@.@@@.@@.@@@@@@@@@@@@@@.@@@..@@@@.@@@..@@@.@.@@.@.@.@@@..@@@@@@.@.@.@.@.@@@@@@@@@@@.@@@@.@...@@@@@@@@@@@.@@...@@@@@@@@...\n@@@@.@.@@@.@..@.@@@@@@@@@@@@@@.@..@@@..@@@.@.@.@@@...@@@@@..@.@.@.@@@..@.@@...@..@.@....@@@@@.@..@@@.@@@@@@@@.@@.@@@.@..@.@@@@@@@@.@@.@@@..\n@.@@.@@@@@@@@@@.@..@...@@@.@@@@@@.@.@@@@@.@@@.@@@...@..@.@.@.@@@@.@@.@.@@@@@@@..@@...@@@@@@@.@.@@@@.@@.@@..@@.@..@@.@@@.@@@@@.@@@..@@.@..@@\n@.@@@...@.@@@@@..@@@@@@..@.....@@.@.@@@@...@@@.@@@.@@@@.@@@@@@@@@..@@@@@@..@@@@@..@@@@.@@@@@..@@.@...@..@@@..@@@.@@@.@@@.@.@..@@@.@@@@@@@.@\n@@@@...@@.@@

In [6]:
input_test_raw = open_input( suffix="test")
input_test_raw

'..@@.@@@@.\n@@@.@.@.@@\n@@@@@.@.@@\n@.@@@@..@.\n@@.@@@@.@@\n.@@@@@@@.@\n.@.@.@.@@@\n@.@@@.@@@@\n.@@@@@@@@.\n@.@.@@@.@.'

In [7]:
from ast import Tuple


def preprocess_data (data):
    # dtype='U10'
    rotations = []
    for row in data.split("\n"):   
        rotations.append([a for a in row])

    return np.array(rotations)
test_data = preprocess_data(input_test_raw)
input_data = preprocess_data(input_raw)
display(test_data)

array([['.', '.', '@', '@', '.', '@', '@', '@', '@', '.'],
       ['@', '@', '@', '.', '@', '.', '@', '.', '@', '@'],
       ['@', '@', '@', '@', '@', '.', '@', '.', '@', '@'],
       ['@', '.', '@', '@', '@', '@', '.', '.', '@', '.'],
       ['@', '@', '.', '@', '@', '@', '@', '.', '@', '@'],
       ['.', '@', '@', '@', '@', '@', '@', '@', '.', '@'],
       ['.', '@', '.', '@', '.', '@', '.', '@', '@', '@'],
       ['@', '.', '@', '@', '@', '.', '@', '@', '@', '@'],
       ['.', '@', '@', '@', '@', '@', '@', '@', '@', '.'],
       ['@', '.', '@', '.', '@', '@', '@', '.', '@', '.']], dtype='<U1')

In [12]:
def solution(data):
    solution = 0
    
    row_max = data.shape[0]
    col_max = data.shape[1]
    
    def check_neighbors(i, j):
        num_adjacent = 0
        
        for di in range(-1,2):
            for dj in range(-1,2):
                if (di!=0 or dj!=0) and 0 <= i+di < row_max and 0 <= j+dj < col_max:
                    if data[i+di, j+dj] == '@':
                        num_adjacent += 1
                    
        return num_adjacent
    
    for i in range(data.shape[0]):
        for j in range(data.shape[1]):
            if data[i,j] == '@':
                num_adj = check_neighbors(i,j)
                if num_adj < 4:
                    solution += 1

            
    return solution

sol_test = solution(test_data)
print("Test Solution:", sol_test)
sol_input = solution(input_data)
print("Input Solution:", sol_input)

Test Solution: 13
Input Solution: 1451


In [16]:
def solution2(data):
    solution = 0
    
    row_max = data.shape[0]
    col_max = data.shape[1]
    
    data_cpy = np.copy(data)
    
    def check_neighbors(i, j):
        num_adjacent = 0
        
        for di in range(-1,2):
            for dj in range(-1,2):
                if (di!=0 or dj!=0) and 0 <= i+di < row_max and 0 <= j+dj < col_max:
                    if data_cpy[i+di, j+dj] == '@':
                        num_adjacent += 1
                    
        return num_adjacent
    
    stop = False
    while not stop:
        changes = []
        
        for i in range(data_cpy.shape[0]):
            for j in range(data_cpy.shape[1]):
                if data_cpy[i,j] == '@':
                    num_adj = check_neighbors(i,j)
                    if num_adj < 4:
                        solution += 1
                        changes.append((i,j))
        
        for (i,j) in changes:
            data_cpy[i,j] = '.'
        
        if not changes:
            stop = True
        
    return solution

sol_test = solution2(test_data)
print("Test Solution:", sol_test)
sol_input = solution2(input_data)
print("Input Solution:", sol_input)

Test Solution: 43
Input Solution: 8701
